# Ejercicio 12: Web Scraping

## Objetivo de la práctica

El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.

### Parte 0: Planificar
1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.

Librerias necesarias

In [35]:
import os
import requests
import xml.etree.ElementTree as ET
import re
import fitz 
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

from dotenv import load_dotenv

import faiss

## Parte 1: Entender el sitio web objetivo

- Analizar la estructura de la página web a ser analizada.
- Identificar los elementos HTML que contienen los datos bsuscados.

In [3]:

file = 'page_499.html'

# Load the HTML file
with open(file, "r", encoding="utf-8") as file:
    html_content = file.read()
    
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [4]:
# Extracting the recipe title
title = soup.find("meta", {"property": "og:title"})["content"]
title

'Independence Day Cookies'

In [5]:
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

4 cups all-purpose flour
1 teaspoon baking soda
1 teaspoon salt
½ teaspoon baking powder
2 cups butter-flavored shortening
1 cup white sugar
1 cup brown sugar
2 large eggs
1 (12 ounce) package white chocolate chips
1 (12 ounce) package dried blueberries
2 (6 ounce) packages dried cranberries


Llenar de paginas el txt recetas

In [7]:
# Cargar el archivo .xml
tree = ET.parse('maping.xml')
root = tree.getroot()
print(root)

# Es la ruta del root
namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

# Regex para obtener solo las urls de recetas
pattern = re.compile(r'^https://www\.allrecipes\.com/recipe/\d+/.+/$')

# Buscar desde la raíz guardada en namespace, todos los elementos con url
urls = root.findall('ns:url/ns:loc', namespaces=namespace)

# Limiar las url que se guardan
max_urls = 100
count = 1

# Escribir en urls.txt
with open('urls.txt', 'w', encoding='utf-8') as file:
    for url in urls:
        url_text = url.text.strip()
        if pattern.match(url_text):
            file.write(url_text + '\n')
            count += 1
            if count >= max_urls:
                break  # salir cuando llegue a max_urls

<Element '{http://www.sitemaps.org/schemas/sitemap/0.9}urlset' at 0x0000024DFFD750D0>


## Parte 2: Obtener los datos deseados

* Buscar dentro del contenido del txt y extraer al informacion

In [21]:
# Dirección del txt de donde lee y la carpeta donde guarda
urls_file = "urls.txt"
output_dir = "html_guardados"

# Para que el servidor pinese que la solicitud es de navegador (no funcionó sin esto)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}
# Usar el encabezad http
with open(urls_file, "r", encoding="utf-8") as f:
    url_list = [line.strip() for line in f if line.strip()]

# Recorrer las url
for i, url in enumerate(url_list, start=1):
    try:
        response = requests.get(url, headers=headers, timeout=10) # Hacer solicitud
        response.raise_for_status()                               # Si la solicitud da error, tirar excepción 
        file_name = os.path.join(output_dir, f"page_{i}.html")    # Crear el archivo donde se va a guardar
        # Guardar el archivo
        with open(file_name, "w", encoding="utf-8") as f:         
            f.write(response.text)
        print(f"Saved: {file_name}")
    except Exception as e:
        print(f"Error with {url}: {e}")


Saved: html_guardados\page_1.html
Saved: html_guardados\page_2.html
Saved: html_guardados\page_3.html
Saved: html_guardados\page_4.html
Saved: html_guardados\page_5.html
Saved: html_guardados\page_6.html
Saved: html_guardados\page_7.html
Saved: html_guardados\page_8.html
Saved: html_guardados\page_9.html
Saved: html_guardados\page_10.html
Saved: html_guardados\page_11.html
Saved: html_guardados\page_12.html
Saved: html_guardados\page_13.html
Saved: html_guardados\page_14.html
Saved: html_guardados\page_15.html
Saved: html_guardados\page_16.html
Saved: html_guardados\page_17.html
Saved: html_guardados\page_18.html
Saved: html_guardados\page_19.html
Saved: html_guardados\page_20.html
Saved: html_guardados\page_21.html
Saved: html_guardados\page_22.html
Saved: html_guardados\page_23.html
Saved: html_guardados\page_24.html
Saved: html_guardados\page_25.html
Saved: html_guardados\page_26.html
Saved: html_guardados\page_27.html
Saved: html_guardados\page_28.html
Saved: html_guardados\page_29

Con los html podemos extraer la informacion deseada

In [22]:

def extract_information(html):
    soup = BeautifulSoup(html, "html.parser")

    # Recuperar título del platillo
    title = soup.find("meta", {"property": "og:title"})["content"]

    # Recuperar descripción del platillo
    description = soup.find("meta", {"property": "og:description"})["content"]

    # Recuperar los ingredientes
    ingredient_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
    ingredients = [ingredient.text.strip() for ingredient in ingredient_section]

    # Recuperar la valoración del platillo
    valoration_section = soup.find("div", attrs={
        "data-tracking-category": "User Recipe Action",
        "class": "comp mm-recipes-review-bar__rating mntl-text-block text-label-300"
    })
    valoration = valoration_section.text.strip() if valoration_section else None

    # Inicializar variables de time y servings
    time = None
    servings = None

    # Buscar 'Total Time' y 'Servings'
    for item in soup.select(".mm-recipes-details__item"):
        label = item.select_one(".mm-recipes-details__label").text.strip().rstrip(':')
        value = item.select_one(".mm-recipes-details__value").text.strip()

        if label == "Total Time":
            time = value
        if label == "Servings":
            servings = value

    return {
        "title": title,
        "description": description,
        "ingredients": ingredients,
        "valoration": valoration,
        "time": time,
        "servings": servings
    }


In [23]:
folder_path = "./html_guardados"

# Guardar el corpus
corpus = []

# Iterar sobre cada archivo .html
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            html = file.read()
            info = extract_information(html)
            corpus.append(info)
    except Exception as e:
        print(f"Error en {filename}: {e}")

# Crear dataframe 
df = pd.DataFrame(corpus)

df

,title,description,ingredients,valoration,time,servings
0,Grilled Italian Sausage,A cast iron skillet is all you'll need to make...,"[1 (12 ounce) package Italian sausage links, 2...",4.0,25 mins,3
1,Watermelon and Tomato Feta Salad,This watermelon and tomato feta salad recipe i...,"[1 (4 pound) seedless watermelon, peeled and c...",4.1,20 mins,8
2,Carrot Ginger Mules,This carrot ginger mule recipe adds a refreshi...,"[4 cups carrot juice, 1 cup pineapple juice, ¼...",5.0,1 day 10 mins,8
3,Thai Chicken Thigh Kebabs,These colorful chicken kebabs are a great summ...,"[⅓ cup soy sauce, 2 tablespoons honey, 1 table...",5.0,4 hrs 30 mins,6
4,Curry Chicken Pasta Salad,This curry-flavored pasta salad with chicken r...,"[5 chicken breasts, 1 (8 ounce) package pasta ...",4.2,3 hrs 35 mins,6
...,...,...,...,...,...,...
94,Easy Grilled Chicken Wings,These easy grilled chicken wings are perfect f...,"[20 chicken wings, 2 tablespoons olive oil, o...",5.0,40 mins,20
95,Apricot Lavender Jam,This delicious apricot jam with lavender makes...,"[4 ½ cups white sugar, divided, 1 tablespoon d...",5.0,1 day 8 hrs 55 mins,128
96,Summer Strawberry Buckle,This simple strawberry buckle is the ultimate ...,"[¾ cup white sugar, ¼ cup butter, softened, 1 ...",4.5,45 mins,8
97,Chef John's Stuffed Peppers,This is the best stuffed peppers recipe that s...,"[1 cup uncooked long grain white rice, 2 cups ...",4.8,1 hr 50 mins,8


## Parte 3: Obtener enlaces relacionados
* Encontrar links a otras recetas para completar el corpus

In [17]:
with open('urls.txt', 'r', encoding='utf-8') as urls_file:
    contenido = urls_file.read()
print(contenido)

https://www.allrecipes.com/recipe/272991/grilled-italian-sausage/
https://www.allrecipes.com/recipe/272967/memorial-day-baked-beans/
https://www.allrecipes.com/recipe/272979/picnic-macaroni-salad/
https://www.allrecipes.com/recipe/273632/black-raspberry-sorbet/
https://www.allrecipes.com/recipe/272934/red-white-and-blue-pie/
https://www.allrecipes.com/recipe/272936/best-4th-of-july-patriotic-brownies/
https://www.allrecipes.com/recipe/272933/rhuberry-crisp/
https://www.allrecipes.com/recipe/217016/nancys-boiled-gazpacho/
https://www.allrecipes.com/recipe/272671/my-favorite-grilled-chicken-ever/
https://www.allrecipes.com/recipe/215580/watermelon-and-tomato-feta-salad/
https://www.allrecipes.com/recipe/270841/carrot-ginger-mules/
https://www.allrecipes.com/recipe/272914/thai-chicken-thigh-kebabs/
https://www.allrecipes.com/recipe/232629/curry-chicken-pasta-salad/
https://www.allrecipes.com/recipe/272597/mexican-street-corn-dip/
https://www.allrecipes.com/recipe/27070/new-red-potato-sala

## Parte 4: Hacer RAG con las recetas obtenidas
* Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus

Preprocesar el corpus cargado

In [24]:
df['raw'] = df['title'] + '. ' + df['description'] + ' Ingredients: ' + df['ingredients'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))

df

,title,description,ingredients,valoration,time,servings,raw
0,Grilled Italian Sausage,A cast iron skillet is all you'll need to make...,"[1 (12 ounce) package Italian sausage links, 2...",4.0,25 mins,3,Grilled Italian Sausage. A cast iron skillet i...
1,Watermelon and Tomato Feta Salad,This watermelon and tomato feta salad recipe i...,"[1 (4 pound) seedless watermelon, peeled and c...",4.1,20 mins,8,Watermelon and Tomato Feta Salad. This waterme...
2,Carrot Ginger Mules,This carrot ginger mule recipe adds a refreshi...,"[4 cups carrot juice, 1 cup pineapple juice, ¼...",5.0,1 day 10 mins,8,Carrot Ginger Mules. This carrot ginger mule r...
3,Thai Chicken Thigh Kebabs,These colorful chicken kebabs are a great summ...,"[⅓ cup soy sauce, 2 tablespoons honey, 1 table...",5.0,4 hrs 30 mins,6,Thai Chicken Thigh Kebabs. These colorful chic...
4,Curry Chicken Pasta Salad,This curry-flavored pasta salad with chicken r...,"[5 chicken breasts, 1 (8 ounce) package pasta ...",4.2,3 hrs 35 mins,6,Curry Chicken Pasta Salad. This curry-flavored...
...,...,...,...,...,...,...,...
94,Easy Grilled Chicken Wings,These easy grilled chicken wings are perfect f...,"[20 chicken wings, 2 tablespoons olive oil, o...",5.0,40 mins,20,Easy Grilled Chicken Wings. These easy grilled...
95,Apricot Lavender Jam,This delicious apricot jam with lavender makes...,"[4 ½ cups white sugar, divided, 1 tablespoon d...",5.0,1 day 8 hrs 55 mins,128,Apricot Lavender Jam. This delicious apricot j...
96,Summer Strawberry Buckle,This simple strawberry buckle is the ultimate ...,"[¾ cup white sugar, ¼ cup butter, softened, 1 ...",4.5,45 mins,8,Summer Strawberry Buckle. This simple strawber...
97,Chef John's Stuffed Peppers,This is the best stuffed peppers recipe that s...,"[1 cup uncooked long grain white rice, 2 cups ...",4.8,1 hr 50 mins,8,Chef John's Stuffed Peppers. This is the best ...


In [25]:
# Diccionario de fracciones Unicode a float
unicode_fractions = {
    '½': 0.5,
    '⅓': 1/3,
    '⅔': 2/3,
    '¼': 0.25,
    '¾': 0.75,
    '⅕': 0.2,
    '⅖': 0.4,
    '⅗': 0.6,
    '⅘': 0.8,
    '⅙': 1/6,
    '⅚': 5/6,
    '⅛': 0.125,
    '⅜': 0.375,
    '⅝': 0.625,
    '⅞': 0.875,
}

# Función para convertir números fraccionarios en unicode a decimales
def convert_unicode_fractions(text):
    def replace_match(match):
        parts = match.group(0).split()
        if len(parts) == 2:
            # Ej. "4 ½"
            whole = int(parts[0])
            fraction = unicode_fractions.get(parts[1], 0)
            decimal = whole + fraction
        else:
            # Ej. "½"
            decimal = unicode_fractions.get(parts[0], 0)
        
        return f"{round(decimal, 2):.2f}"  # redondeo y formato a 2 decimales

    # Buscar patrones como "4 ½" o "½"
    pattern = re.compile(r'(\d+\s)?[{}]'.format(''.join(unicode_fractions.keys())))
    return pattern.sub(replace_match, text)

In [26]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_doc(doc):
    tokens = regexp_tokenize(doc.lower(), r'\w+')                       # tokenizar
    tokens = [token for token in tokens if token not in stop_words]     # eliminar stopwords
    tokens = [lemmatizer.lemmatize(t) for t in tokens]                  # Lematización
    return ' '.join(tokens)

In [27]:
df["raw_w/o_unicode"] = df["raw"].apply(convert_unicode_fractions)
df["preprocessed"] = df["raw_w/o_unicode"].apply(preprocess_doc)
df

,title,description,ingredients,valoration,time,servings,raw,raw_w/o_unicode,preprocessed
0,Grilled Italian Sausage,A cast iron skillet is all you'll need to make...,"[1 (12 ounce) package Italian sausage links, 2...",4.0,25 mins,3,Grilled Italian Sausage. A cast iron skillet i...,Grilled Italian Sausage. A cast iron skillet i...,grilled italian sausage cast iron skillet need...
1,Watermelon and Tomato Feta Salad,This watermelon and tomato feta salad recipe i...,"[1 (4 pound) seedless watermelon, peeled and c...",4.1,20 mins,8,Watermelon and Tomato Feta Salad. This waterme...,Watermelon and Tomato Feta Salad. This waterme...,watermelon tomato feta salad watermelon tomato...
2,Carrot Ginger Mules,This carrot ginger mule recipe adds a refreshi...,"[4 cups carrot juice, 1 cup pineapple juice, ¼...",5.0,1 day 10 mins,8,Carrot Ginger Mules. This carrot ginger mule r...,Carrot Ginger Mules. This carrot ginger mule r...,carrot ginger mule carrot ginger mule recipe a...
3,Thai Chicken Thigh Kebabs,These colorful chicken kebabs are a great summ...,"[⅓ cup soy sauce, 2 tablespoons honey, 1 table...",5.0,4 hrs 30 mins,6,Thai Chicken Thigh Kebabs. These colorful chic...,Thai Chicken Thigh Kebabs. These colorful chic...,thai chicken thigh kebab colorful chicken keba...
4,Curry Chicken Pasta Salad,This curry-flavored pasta salad with chicken r...,"[5 chicken breasts, 1 (8 ounce) package pasta ...",4.2,3 hrs 35 mins,6,Curry Chicken Pasta Salad. This curry-flavored...,Curry Chicken Pasta Salad. This curry-flavored...,curry chicken pasta salad curry flavored pasta...
...,...,...,...,...,...,...,...,...,...
94,Easy Grilled Chicken Wings,These easy grilled chicken wings are perfect f...,"[20 chicken wings, 2 tablespoons olive oil, o...",5.0,40 mins,20,Easy Grilled Chicken Wings. These easy grilled...,Easy Grilled Chicken Wings. These easy grilled...,easy grilled chicken wing easy grilled chicken...
95,Apricot Lavender Jam,This delicious apricot jam with lavender makes...,"[4 ½ cups white sugar, divided, 1 tablespoon d...",5.0,1 day 8 hrs 55 mins,128,Apricot Lavender Jam. This delicious apricot j...,Apricot Lavender Jam. This delicious apricot j...,apricot lavender jam delicious apricot jam lav...
96,Summer Strawberry Buckle,This simple strawberry buckle is the ultimate ...,"[¾ cup white sugar, ¼ cup butter, softened, 1 ...",4.5,45 mins,8,Summer Strawberry Buckle. This simple strawber...,Summer Strawberry Buckle. This simple strawber...,summer strawberry buckle simple strawberry buc...
97,Chef John's Stuffed Peppers,This is the best stuffed peppers recipe that s...,"[1 cup uncooked long grain white rice, 2 cups ...",4.8,1 hr 50 mins,8,Chef John's Stuffed Peppers. This is the best ...,Chef John's Stuffed Peppers. This is the best ...,chef john stuffed pepper best stuffed pepper r...


Embedding para entrenar el modelo con nuestro corpus de recetas

In [28]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [29]:
query = "pepperoni pizza"
query_emb = model.encode(query)
print(query_emb)

[-1.11345612e-01  4.19089831e-02 -4.07155789e-02  8.55102688e-02
 -6.86945170e-02  2.94043645e-02  9.83207300e-03  4.03417498e-02
  1.40147079e-02 -1.23610377e-01  5.37547730e-02 -9.69437808e-02
  3.45474407e-02  1.31831085e-02  2.65819710e-02 -8.78858417e-02
  1.04220897e-01  3.53136123e-03  2.85468413e-03 -7.79617578e-03
 -8.13834444e-02 -4.84555773e-02 -4.30481732e-02 -4.60941577e-03
  5.44216894e-02  1.38844192e-01  4.71816175e-02  3.81750502e-02
 -3.88631411e-02 -4.86818887e-02 -1.02784624e-02  4.66574244e-02
  1.21300027e-01  9.57046822e-03 -2.98522972e-03 -1.25743458e-02
  5.94118237e-02 -4.19300981e-02  1.40979096e-01 -9.82428342e-03
  2.74148565e-02 -1.25679867e-02  9.49107558e-02 -6.40578419e-02
  8.01342577e-02  2.29160208e-02 -6.19955594e-03 -1.13470750e-02
  2.03015972e-02  1.77209619e-02 -5.15672304e-02  3.87585871e-02
 -1.82473827e-02  2.42548101e-02  2.59807222e-02 -7.13633820e-02
  6.92288252e-03 -7.29552191e-03  2.36795284e-02  1.20053980e-02
 -2.32913531e-02 -1.26494

In [30]:
embeddings = model.encode(df['preprocessed'].tolist())
df['embeddings'] = embeddings.tolist()
print(embeddings)

[[-1.12937488e-01 -1.68924648e-02 -1.18400832e-03 ...  2.21417435e-02
  -7.62589797e-02 -1.41918650e-02]
 [-7.68168569e-02 -1.20627380e-03 -1.93780300e-03 ...  9.91578624e-02
   3.01940106e-02 -3.18650715e-03]
 [-6.66522458e-02 -1.38660995e-02 -2.63562594e-02 ...  2.51862761e-02
   4.58522290e-02  1.24638714e-02]
 ...
 [ 2.76574921e-02 -4.41299602e-02  3.44695449e-02 ... -5.52931651e-02
   2.38999957e-03 -7.94358924e-02]
 [-9.42662209e-02  1.87707169e-03  3.21280695e-02 ...  2.02841386e-02
  -2.12419294e-02  2.53611673e-02]
 [-7.97989517e-02 -6.94489703e-02  2.76676994e-02 ...  4.41319831e-02
  -2.35829130e-02 -6.24958921e-05]]


FAISS

In [31]:
dimension = embeddings.shape[1]
print(dimension)

index = faiss.IndexFlatL2(dimension)  # Distancia euclideana

index.add(np.array(embeddings))       # Agregar los embeddings al índice
print(index)

384
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000024DFFAE8420> >


In [32]:
K = 10
D, I = index.search(np.array([query_emb], dtype='float32'), K)

# Solo imprimir para ver que onda
for i in I[0]:
    print(df["title"][i])

print(I)

Rachel's Pizza Pasta Salad
Chef John's Stuffed Peppers
Kelsey's Favorite Stuffed Green Peppers
Greek Watermelon Pizza
Beef and Rice Stuffed Bell Peppers
Stuffed Hot Peppers
Vegan Stuffed Peppers
Yummy Summer Picnic Salad
Stuffed Poblano Peppers
Grilled Italian Sausage
[[41 97 37 90 98 93 36 18 89  0]]


In [43]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
# Objeto tipo cliente de OpenAi
api_key = os.getenv("CHATGPT_API_KEY")
client = OpenAI(api_key=api_key)

In [44]:
print(I[0])
context = "\n\n".join(df.loc[I[0], "raw_w/o_unicode"].values)
print(context)

[41 97 37 90 98 93 36 18 89  0]
Rachel's Pizza Pasta Salad. Pizza toppings like olives, pepperoni, cheese, green peppers, and onions star in this pasta salad, perfect for a potluck or picnic. Ingredients: 1 (16 ounce) package tri-color rigatoni pasta, 2  onions, diced, 3  tomatoes, diced, 12 ounces provolone cheese, cut into strips, 2  green bell peppers, diced, 1 (8 ounce) package sliced pepperoni, 0.25 cup sliced olives, or to taste, 1 cup olive oil, 0.50 cup vinegar, 2 tablespoons grated Parmesan cheese, 1 tablespoon white sugar, 0.50 teaspoon dried oregano, salt and ground black pepper to taste

Chef John's Stuffed Peppers. This is the best stuffed peppers recipe that simply fills bell peppers with a mix of rice, ground beef and pork sausage for an easy yet satisfying supper. Ingredients: 1 cup uncooked long grain white rice, 2 cups water, 1  onion, diced, 1 tablespoon olive oil, 2 cups marinara sauce, 1 cup beef broth, 1 tablespoon balsamic vinegar, 0.25 teaspoon crushed red peppe

In [45]:
prompt = f"""Eres una aplicación de tipo Retrieval-Augmented Generation (RAG) que siempre responde en español.
Tu tarea es ayudar al usuario a encontrar la receta que necesita, utilizando únicamente el contexto proporcionado. 
La respuesta debe incluir el nombre de la receta, una breve descripción y los ingredientes.
Si la información solicitada no se encuentra en el contexto, responde con: "Lo siento, no encontré una receta relacionada con tu consulta."

Contexto:
{context}

Pregunta del usuario:
{query}
"""

In [46]:
response = client.responses.create(
    model="gpt-4.1",
    input= prompt
)

print(response.output_text)

Nombre de la receta: Rachel's Pizza Pasta Salad

Descripción: 
Esta ensalada de pasta combina los ingredientes clásicos de una pizza, como aceitunas, pepperoni, queso, pimientos verdes y cebolla, en una refrescante ensalada fría. Es perfecta para llevar a un picnic o compartir en una reunión.

Ingredientes:
- 1 paquete (16 onzas) de pasta rigatoni tricolor
- 2 cebollas, picadas
- 3 tomates, picados
- 12 onzas de queso provolone, cortado en tiras
- 2 pimientos verdes, picados
- 1 paquete (8 onzas) de pepperoni en rodajas
- 0,25 taza de aceitunas en rodajas, al gusto
- 1 taza de aceite de oliva
- 0,50 taza de vinagre
- 2 cucharadas de queso parmesano rallado
- 1 cucharada de azúcar blanca
- 0,50 cucharadita de orégano seco
- Sal y pimienta negra molida al gusto
